# Project 4: Document Classification

### Importing the data

In [3]:
# Import package
import numpy as np

# Assign filename to variable: file
file_path = '/home/alainkuiete/Documents/DATA620/datasets/spambase.data'

# Load file as array: digits
spam_data = np.loadtxt(file_path, delimiter=',')

# Print datatype of digits
print(type(spam_data))


<class 'numpy.ndarray'>


In [4]:
spam_data.shape

(4601, 58)

### Split into Features and target variables

In [5]:
X = spam_data[:,:-1]
X

array([[0.000e+00, 6.400e-01, 6.400e-01, ..., 3.756e+00, 6.100e+01,
        2.780e+02],
       [2.100e-01, 2.800e-01, 5.000e-01, ..., 5.114e+00, 1.010e+02,
        1.028e+03],
       [6.000e-02, 0.000e+00, 7.100e-01, ..., 9.821e+00, 4.850e+02,
        2.259e+03],
       ...,
       [3.000e-01, 0.000e+00, 3.000e-01, ..., 1.404e+00, 6.000e+00,
        1.180e+02],
       [9.600e-01, 0.000e+00, 0.000e+00, ..., 1.147e+00, 5.000e+00,
        7.800e+01],
       [0.000e+00, 0.000e+00, 6.500e-01, ..., 1.250e+00, 5.000e+00,
        4.000e+01]])

In [6]:
y = spam_data[:,-1]
y

array([1., 1., 1., ..., 0., 0., 0.])


### Processing the data

In [7]:
# Import the necessary modules
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state=42)

# Create the classifier: logreg
logreg = LogisticRegression()

# Fit the classifier to the training data
logreg.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = logreg.predict(X_test)

# Compute and print the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[1017   72]
 [  77  675]]
              precision    recall  f1-score   support

         0.0       0.93      0.93      0.93      1089
         1.0       0.90      0.90      0.90       752

    accuracy                           0.92      1841
   macro avg       0.92      0.92      0.92      1841
weighted avg       0.92      0.92      0.92      1841



/home/alainkuiete/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [8]:
X_test.shape

(1841, 57)

## Using a different spam document to test the model

In [9]:
from nltk.tokenize import word_tokenize
import nltk
import re
def remove_noise(text, stop_words = []):
    tokens = word_tokenize(text)
    cleaned_tokens = []
    for token in tokens:
        token = re.sub('[^A-Za-z0-9]+', '', token)
        if len(token) > 1 and token.lower() not in stop_words:
            # Get lowercase
            cleaned_tokens.append(token.lower())
    return cleaned_tokens
text = remove_noise("It is lovely weather we are having. I hope the weather continues.")

In [10]:
def content_wsw(text):
    stopwords = nltk.corpus.stopwords.words('english')
    content = [w for w in text if w.lower() not in stopwords]
    return content
content_wsw(text)

['lovely', 'weather', 'hope', 'weather', 'continues']

### Loading the spam files from  the directory spam_2

In [11]:
# Assign filename: file
file_path = '/home/alainkuiete/Downloads/spam_2/00001.317e78fa8ee2f54cd4890fdc09ba8176'

with open(file_path, 'r') as file:
    text = file.read()

In [12]:
stopwords = nltk.corpus.stopwords.words('english')
text_tokenized = remove_noise(text, stopwords)

In [13]:
print(len(text_tokenized))

403


In [14]:
# Import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TfidfVectorizer: tfidf
tfidf = TfidfVectorizer(max_features=57)

# Apply fit_transform to document: csr_mat
csr_mat = tfidf.fit_transform(text_tokenized)

# Print result of toarray() method
print(csr_mat.toarray()[:,:57])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [15]:
X_test = csr_mat.toarray()

In [16]:
X_test = X_test.reshape(X_test.shape[0],57)

In [17]:
y_test = np.zeros(X_test.shape[0])

In [18]:
# Predict the labels of the test set: y_pred
y_pred = logreg.predict(X_test)

# Compute and print the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[399   4]
 [  0   0]]
              precision    recall  f1-score   support

         0.0       1.00      0.99      1.00       403
         1.0       0.00      0.00      0.00         0

    accuracy                           0.99       403
   macro avg       0.50      0.50      0.50       403
weighted avg       1.00      0.99      1.00       403



/home/alainkuiete/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


We have a good accuracy for the prediction because the file contain only spams.

References:
Natural Language with python
DataCamp Supervised Learning.Logistic regression